In [ ]:
from model import UNet
from utils import data_loaders
from train import training_loop

import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
root_dir = "data"
train_loader, val_loader, test_loader = data_loaders(root_dir=root_dir)

In [ ]:
model = UNet(in_channels=1, out_channels=1).to(device)
n_epochs = 100
lr = 1e-4
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [ ]:
train_losses, val_losses = training_loop(n_epochs, optimizer, model, loss_fn, train_loader, val_loader, device)